<a href="https://colab.research.google.com/github/caeltarifa/big_data_web_scraping/blob/main/Big_data_storage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Storing crawled data 

In [ ]:
!python -m pip install boto3


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import boto3
import os
import requests
import tqdm

In [ ]:
dynamo_client  =  boto3.resource(service_name = 'data_crawled_dydb',region_name = 'us-east-1',
              aws_access_key_id = 'AKIA3BS5NFXXXXXXX',
              aws_secret_access_key = 'qfGTJL28HrqcbhKCM0t//xxx7gTGG4iNrv3/d94Lsp')

In [ ]:
product_table = dynamo_client.Table('institution')
product_table.table_status

product_table_link = dynamo_client.Table('link_collection')
product_table_link.table_status

product_table_file = dynamo_client.Table('file_collection')
product_table_file.table_status

In [ ]:
dynamo_client.get_available_subresources()

In [ ]:
def query_police_department_record_by_guid(guid):  
    db = dynamodb_resource()
    extra_msg = {"region_name": REGION, "aws_service": "dynamodb", 
        "police_department_table":POLICE_DEPARTMENTS_TABLE,
        "guid":guid}
    log.info(f"Get PD record by GUID", extra=extra_msg)
    pd_table = db.Table(POLICE_DEPARTMENTS_TABLE)
    response = pd_table.get_item(
        Key={
            'guid': guid
            }
    )
    return response['Item']

In [ ]:
print(query_police_department_record_by_guid("jlkdajfldskj1312312"))

# Data crawling with scrapy

In [9]:
!pip install scrapy 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 264 kB 23.1 MB/s 
     |████████████████████████████████| 93 kB 2.1 MB/s 
     |████████████████████████████████| 3.1 MB 55.3 MB/s 
     |████████████████████████████████| 254 kB 47.6 MB/s 
     |████████████████████████████████| 57 kB 4.0 MB/s 
     |████████████████████████████████| 4.0 MB 59.4 MB/s 
     |████████████████████████████████| 74 kB 3.0 MB/s 
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.6-py3-none-any.whl size=11958 sha256=233fa825ab3ec3d29404a4e46aa79b4a13c34195dec6868cf057513cb19bec52
  Stored in directory: /root/.cache/pip/wheels/c9/d6/6a/de198d890277cde60ca3dbebe7ae592d3b381c7d9bb2455f4d
Successfully built PyDispatcher


In [10]:
!scrapy startproject chilean_data_explore

New Scrapy project 'chilean_data_explore', using template directory '/usr/local/lib/python3.7/dist-packages/scrapy/templates/project', created in:
    /content/chilean_data_explore

You can start your first spider with:
    cd chilean_data_explore
    scrapy genspider example example.com


In [16]:
!pwd
import os
os.chdir('/content/chilean_data_explore/chilean_data_explore/spiders')

/content/chilean_data_explore/chilean_data_explore/spiders


In [18]:
#%%writefile -a /chilean_data_explore/chilean_data_explore/spiders/quotes_spider.py
%%writefile -a quotes_spider.py
import scrapy

class QuotesSpider(scrapy.Spider):
  name = 'quotes'

  def start_requests (self):
    urls = [
      'http://quotes.toscrape.com/page/1/',
      'http://quotes.toscrape.com/page/2/',
    ]
    for url in urls:
      yield scrapy.Request(url=url, callback=self.parse)

  def parse(self, response):
    pag=response.url.split('/')[-2]
    filename = f'quotes-{page}.html'
    with open(filename, 'wb') as f:
      f.write(response.body)
    self.log(f'Saved file {filename}' )

Appending to quotes_spider.py


In [19]:
!python quotes_spider.py

In [ ]:
!scrapy shell 'https://quotes.toscrape.com'


2022-10-15 03:33:57 [scrapy.utils.log] INFO: Scrapy 2.6.3 started (bot: chilean_data_explore)
2022-10-15 03:33:57 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.14, cssselect 1.1.0, parsel 1.6.0, w3lib 2.0.1, Twisted 22.8.0, Python 3.7.14 (default, Sep  8 2022, 00:06:44) - [GCC 7.5.0], pyOpenSSL 22.1.0 (OpenSSL 3.0.5 5 Jul 2022), cryptography 38.0.1, Platform Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic
2022-10-15 03:33:57 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'chilean_data_explore',
 'DUPEFILTER_CLASS': 'scrapy.dupefilters.BaseDupeFilter',
 'LOGSTATS_INTERVAL': 0,
 'NEWSPIDER_MODULE': 'chilean_data_explore.spiders',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['chilean_data_explore.spiders']}
2022-10-15 03:33:57 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2022-10-15 03:33:57 [scrapy.extensions.telnet] INFO: Telnet Password: 84d5da1a1de7940a
2022-10-15 03:33:57 [scrapy.middleware] INFO: Enabled extensions:
['sc

#References

> DynamoDB and its purposes

*   [A one size fits all database doesn't fit anyone
](https://www.allthingsdistributed.com/2018/06/purpose-built-databases-in-aws.html)

*   [Amazon DynamoDB](https://aws.amazon.com/dynamodb/)

*   [Scaling globally with the new AWS](https://www.allthingsdistributed.com/2022/08/aws-launches-middle-east-uae-region.html)


> Interfaces for reliable connections

*   [DynamoDB and the AWS SDKs](https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/GettingStarted.html
)

*   [Boto3 and Amazon DynamoDB](https://www.section.io/engineering-education/python-boto3-and-amazon-dynamodb-programming-tutorial/)

*   [DynamoDb in Python using BOTO3](https://www.analyticsvidhya.com/blog/2022/05/working-with-dynamodb-in-python-using-boto3/)

> Scrapy 

* [Google Colab tips: using both %%writefile magic and %%javascript magic in the same cell
](https://stephencowchau.medium.com/google-colab-tips-using-both-writefile-magic-and-javascript-magic-in-the-same-cell-7820e508e455)
